In [18]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import integrate
import geopy.distance
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import requests

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [11]:
cred = credentials.Certificate('./../Backend/serviceAccount.json')
app_firebase = firebase_admin.initialize_app(cred)
db = firestore.client()

In [31]:
import sys
 
sys.path.append('./../Backend/')

In [32]:

 
# importing the add and odd_even
# function
from key_value.token import apiKey, apiUrl, auth_header


 


In [34]:
auth_header

{'x-apikey': 'IaF6zOUaj8VG4ambmUwOodEFaxBnrR3A'}

In [17]:
path = './ONLY_TEST/'
csv_files = glob.glob(os.path.join(path, "*.csv"))
for f in csv_files:
    filename = f.split('/')[2]
    flight_id = filename.split('-')[0]
    print(flight_id)
    doc_ref = db.collection('aircraft_engine').document(f'{flight_id}')
    docs = doc_ref.get()

    if docs.exists:
        print("Found")
        print(docs.to_dict())
    else:
        print("Not found")




AIQ3437
Not found
AIQ3102
Not found
AIQ3252
Not found
NOK570
Not found
NOK100
Not found
AIQ512
Found
{'type': 'A320', 'flight_id': 'AIQ512', 'engine_type': 'jet', 'engine_count': 2}
AIQ3029
Not found
AIQ311
Not found
AIQ3362
Not found
AIQ3188
Not found
NOK340
Not found
NOK122
Not found
TLM702
Not found
